____
__Universidad Tecnológica Nacional, Buenos Aires__<br/>
__Ingeniería Industrial__<br/>
__Cátedra de Ciencia de Datos - Curso I5521 - Turno sabado mañana__<br/>
__Elaborado por: Agustin Velazquez__<br/>
__Editado por: Nicolas Aguirre__

____

# Combinando Datos con Pandas

**Datos** : https://datos.gob.ar/dataset/produccion-archivo-1992-2019-indicadores-sectoriales-acero

**Link util** : https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.display import Image

Tenemos datos distribuidos en 4 datasets. Queremos trabajar con ellos en conjunto. Importamos los 4 datasets y procederemos a combinarlos para generar nuevas tablas.

In [ ]:
expo = pd.read_csv('acero-exportaciones-series.csv')
impo = pd.read_csv('acero-importaciones-series.csv')
prod = pd.read_csv('acero-produccion-series.csv')
consumo = pd.read_csv('acero-consumo-series.csv')

In [ ]:
# Renombremos columnas para que luego cuando grafiquemos nos sea representativo del DF del que provienen
prod = prod.rename(columns={'valor':'acero_produccion'})
# observamos cada dataframe
prod.head(2)

In [ ]:
# Renombremos columnas
impo = impo.rename(columns={'acero_moldeado':'acero_importaciones_acero',
                           'barras_forjadas': 'acero_importaciones_barras',
                           'laminados_largos': 'acero_importaciones_laminados_largos',
                            'laminados_planos': 'acero_importaciones_laminados_planos',
                            'rieles': 'acero_importaciones_rieles',
                            'tubos_con_costura':'acero_importaciones_tubos_con_costura',
                            'tubos_sin_costura': 'acero_importaciones_tubos_sin_costura'                            
                           })

# observamos cada dataframe
impo.head(2)

In [ ]:
# Renombremos columnas
consumo = consumo.rename(columns={'laminados_largos':'acero_consumo_tubos_sin_costura',
                           'laminados_planos': 'acero_consumo_laminados_planos',
                           'tubos_sin_costura': 'acero_consumo_laminados_largos'                           
                           })

# observamos cada dataframe
consumo.head(2)

In [ ]:
# observamos cada dataframe
consumo.head(2)

In [ ]:
# Existe una diferencia en unidades, la produccion esta en toneladas y las importaciones en miles de toneladas
# Transformamos los datos...
prod.acero_produccion = prod.acero_produccion/1000

Observamos como queda la nueva columna de "acero_produccion" del dataframe "prod" luego de haber transformado

In [ ]:
prod.head()

In [ ]:
expo = expo.rename(columns={'acero_moldeado': 'acero_exportaciones_acero',
                            'laminados_largos' :'acero_exportaciones_laminados_largos',
                            'laminados_planos': 'acero_exportaciones_laminados_planos',
                            'rieles': 'acero_exportaciones_rieles',
                            'tubos_con_costura': 'acero_exportaciones_tubos_con_costura',
                            'tubos_sin_costura':'acero_exportaciones_tubos_sin_costura',                           
    
})

In [ ]:
expo

# Trabajar con Fechas

utilizaremos el comando **pd.to_datetime(columna)** para re-escribir la columna "indice_tiempo" que estaba inicialmente como string.

- a la columna transformada en datos con formato de fecha podemos aplicarle .dt.year o .dt.month o dt.day para obtener año, mes o dia de esa columna.

In [ ]:
#Unificamos los datos para tener mediciones por año
expo.indice_tiempo = pd.to_datetime(expo.indice_tiempo) #### indicar que columna queremos transformar #####
expo['year'] = expo.indice_tiempo.dt.year #creamos una nueva columna "year" con .dt.year sobre la col "indice_tiempo"
expo.head()

Applicamos un .groupby() sobre "year" y creamos un nuevo dataframe llamado "expo_by_year"

## Groupby [year]

###  Exportaciones

In [ ]:
#Vamos a trabajar unicamente de aceros laminados 
#Sumamos la cantidad de exportaciones por año
expo_by_year = expo.groupby(["year"])["acero_exportaciones_laminados_largos", "acero_exportaciones_laminados_planos"].sum()
expo_by_year.head()

**PLOT**

con el comando .plot() sobre un dataframe, **si las columnas son numéricas y representan lo mismo** podemos graficarlas en un mismo grafico. Esto es una alternativa rapida a matplotlib y seaborn

In [ ]:
sns.set_context("talk", font_scale=0.8)
expo_by_year.plot(title = "Exportacion de Acero en Argentina en miles de toneladas" )
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

¿que observamos de este grafico?

### Importanciones

In [ ]:
#Hacemos lo mismo con las importaciones ahora
impo.indice_tiempo = pd.to_datetime(impo.indice_tiempo)##### transformamos una columna de fechas a "datetime"
impo['year'] = impo.indice_tiempo.dt.year#### generamos una columna nueva para guardar los años

In [ ]:
impo_by_year = impo.groupby(["year"])[["acero_importaciones_laminados_largos", "acero_importaciones_laminados_planos"]].sum()#### hacemos un groupby de la columna "year" que creamos para "acero_importaciones_laminados_largos","acero_importaciones_laminados_planos"
impo_by_year.head()

**PLOT**

In [ ]:
#Graficamos el nuevo dataframe generado "impo_by_year"
impo_by_year.plot(title = "Importacion de Acero en Argentina en miles de toneladas")
plt.show()

# Concatenar DataFrames

**Como hacemos si queremos tener todos los datos en un mismo grafico?**

[**pandas.concat()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html) !


- Realizaremos un concat de *expo_by_year* y de *impo_by_year*

- Beberemos determinar si concatenar por columnas o filas (axis)

- Determinar si es inner or outter join

In [ ]:
expo_and_impo_by_year = pd.concat([expo_by_year,impo_by_year],
                                  axis = 1, #concatncatenamos columnas
                                  join = "inner", 
                                  ignore_index = False) # Nuestros indices son los años
expo_and_impo_by_year.head(10)

Ahora podemos visualizar todos los datos en una misma tabla

In [ ]:
expo_and_impo_by_year.plot()
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

## Opcion axis y join 

Hay dos fomas de concatenar 

* una por las columnas (axis = 1) 

* otra por las filas (axis = 0), 

por default es cero, que pasa si cambiamos ese valor?

![Axis diagram](https://www.dataquest.io/wp-content/uploads/2019/01/axis_diagram.jpg)

Al igual que en bases de datos ralacionales (ejemplo: SQL), Pandas tiene la opcion de realizar **merge** y **join**.

    - Joins solo puede realizarse de a dos tablas.
    
    - Concat va a funcionar cuando tenemos tablas de la misma altura (numero de filas)
    
    - "Key" es la columna que tienen en comun.
    
    - Con pd.concat() podemos usar inner o outer join
    
    - Existen 4 tipos de join:  

![Inner join](https://www.dataquest.io/wp-content/uploads/2017/12/inner-join.png)
![Left join](https://www.dataquest.io/wp-content/uploads/2019/01/left-join.png)
![Right join](https://www.dataquest.io/wp-content/uploads/2019/01/right-join.png)
![Full outer join](https://www.dataquest.io/wp-content/uploads/2019/01/full-outer-join.png)

# Merge DataFrame

Es muy **recomendando** leer detenidamente los documentos de Merge y su tutorial

Link: https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging

**Ahora vamos a sumarle nuestros datos de Produccion**

In [ ]:
# Modificamos nuestro data-set de produccion para tenerlo de la misma forma que IMPO o EXPO
prod.indice_tiempo =   pd.to_datetime(prod.indice_tiempo)
# creamos la columna "year"
prod['year'] =  prod.indice_tiempo.dt.year 
#sumamos la produccion por cada año
prod_by_year = prod.groupby(["year"])["acero_produccion"].sum().to_frame() 
# Este data-set incluye datos del 2019!
prod_by_year.head(10)

## [Expo_Impo] U [Produccion] 

In [ ]:
#Unimos los dos datasets "expo_and_impo_by_year" y "prod_by_year"
expo_impo_prod_by_year = pd.merge(expo_and_impo_by_year,
                                  prod_by_year,
                                  how='left',
                                  left_index = True,
                                  right_index = True)
expo_impo_prod_by_year.head(40)

# Join Dataframe
 
 **Otra forma de usar merge es con el comando df.join() directamente**

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html

# Append
**En caso de querer agregar mas filas en data-sets similares podemos utilizar pd.append()**

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html

### Conclusion ...

[**DataFrame.append()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.append.html): un metodo rapido para agregar filas pero no columnas!

[**DataFrame.join()**](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html): metodo rapido para hacer join pero solo por index (podriamos haberlo usada aca)

[**pd.concat()**](https://pandas.pydata.org/docs/reference/api/pandas.concat.html): multiproposito y puede ser usado para combinar data-sets por filas o columnas 

[**pd.merge()**](https://pandas.pydata.org/docs/reference/api/pandas.merge.html): util cuando tenemos que unir data-sets con una columna en comun (key). Es la funcion mas *general*. Entendiendo esta funcion y como configurarla podemos obtener los mismo resultados que todas las anteriores.

# Tarea... (?)

1- Unir el data-set de consumo al ultimo creado para poder graficar todo junto. (tener en cuenta las columnas usadas)

2- Obtener un data-set a partir del punto 1 con cuatro columnas correspondientes a EXPO, IMPO, PROD y CONSUM.

        (Puedo sumar columnas a = b + c y crear un DF, puedo usar pd.sum() o crear columnas y luego hacer drop)
        
3- Graficar el data-set del punto 2 con la libreria sea-born y editando titulos, leyendas, colores, etc. 


# **Preguntas ?**





![Axis diagram](https://i.pinimg.com/originals/ec/bf/1c/ecbf1c0337f0623160ac56c89d64d601.jpg)